In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/28 22:43:56 WARN Utils: Your hostname, BOLD-2018144 resolves to a loopback address: 127.0.1.1; using 172.29.7.81 instead (on interface eth0)
22/02/28 22:43:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/28 22:43:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/28 22:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/28 22:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/02/28 22:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
spark.version

'3.0.3'

In [6]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-25 23:32:51--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.248.100
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.248.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  2.37MB/s    in 5m 23s  

2022-02-25 23:38:14 (2.17 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [5]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [9]:
!head -n 1001 fhvhv_tripdata_2021-02.csv > head_HW.csv

In [11]:
import pandas as pd
df_pandas=pd.read_csv('head_HW.csv')

In [12]:
df_pandas.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02764,2021-02-01 00:10:40,2021-02-01 00:21:09,35,39,NaN
1,HV0003,B02764,2021-02-01 00:27:23,2021-02-01 00:44:01,39,35,NaN
2,HV0005,B02510,2021-02-01 00:28:38,2021-02-01 00:38:27,39,91,NaN
3,HV0005,B02510,2021-02-01 00:43:37,2021-02-01 01:23:20,91,228,NaN
4,HV0003,B02872,2021-02-01 00:08:42,2021-02-01 00:17:57,126,250,NaN


In [14]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [6]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [7]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [8]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,StringType,true)))

In [9]:
df = df.repartition(24)

In [10]:
df.write.parquet('fhvhv/2021/02/')

22/02/28 22:47:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/28 22:48:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/28 22:48:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [15]:
!ls -lh fhvhv/2021/02

total 206M
-rw-r--r-- 1 loliveira loliveira    0 Feb 28 22:48 _SUCCESS
-rw-r--r-- 1 loliveira loliveira 8.6M Feb 28 22:48 part-00000-6ae67bf8-5b18-4a0a-8ebf-566a16cd360e-c000.snappy.parquet
-rw-r--r-- 1 loliveira loliveira 8.6M Feb 28 22:48 part-00001-6ae67bf8-5b18-4a0a-8ebf-566a16cd360e-c000.snappy.parquet
-rw-r--r-- 1 loliveira loliveira 8.6M Feb 28 22:48 part-00002-6ae67bf8-5b18-4a0a-8ebf-566a16cd360e-c000.snappy.parquet
-rw-r--r-- 1 loliveira loliveira 8.6M Feb 28 22:48 part-00003-6ae67bf8-5b18-4a0a-8ebf-566a16cd360e-c000.snappy.parquet
-rw-r--r-- 1 loliveira loliveira 8.6M Feb 28 22:48 part-00004-6ae67bf8-5b18-4a0a-8ebf-566a16cd360e-c000.snappy.parquet
-rw-r--r-- 1 loliveira loliveira 8.6M Feb 28 22:48 part-00005-6ae67bf8-5b18-4a0a-8ebf-566a16cd360e-c000.snappy.parquet
-rw-r--r-- 1 loliveira loliveira 8.6M Feb 28 22:48 part-00006-6ae67bf8-5b18-4a0a-8ebf-566a16cd360e-c000.snappy.parquet
-rw-r--r-- 1 loliveira loliveira 8.6M Feb 28 22:48 part-00007-6ae67bf8-5b18-4a0a-8ebf-566a16cd36

In [16]:
df=spark.read.parquet('fhvhv/2021/02/')

In [17]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [18]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [20]:
from pyspark.sql import functions as F

In [23]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('hvfhs_license_num', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID').show()

+-----------------+-----------+------------+------------+------------+
|hvfhs_license_num|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------------+-----------+------------+------------+------------+
|           HV0005| 2021-02-04|  2021-02-04|          81|          81|
|           HV0003| 2021-02-03|  2021-02-03|          29|         154|
|           HV0003| 2021-02-03|  2021-02-03|         203|         203|
|           HV0005| 2021-02-02|  2021-02-02|          91|         179|
|           HV0003| 2021-02-03|  2021-02-03|         126|         116|
|           HV0003| 2021-02-04|  2021-02-04|          56|           9|
|           HV0003| 2021-02-04|  2021-02-04|          81|          53|
|           HV0003| 2021-02-04|  2021-02-04|          18|         168|
|           HV0005| 2021-02-04|  2021-02-04|         136|         116|
|           HV0005| 2021-02-04|  2021-02-04|           7|         159|
|           HV0003| 2021-02-04|  2021-02-04|         122|         101|
|     

In [24]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-02-04 06:56:20|2021-02-04 06:58:25|          81|          81|   null|
|           HV0003|              B02876|2021-02-03 20:04:21|2021-02-03 20:19:09|          29|         154|   null|
|           HV0003|              B02880|2021-02-03 07:21:31|2021-02-03 07:29:04|         203|         203|   null|
|           HV0005|              B02510|2021-02-02 11:30:49|2021-02-02 12:11:16|          91|         179|   null|
|           HV0003|              B02879|2021-02-03 18:54:27|2021-02-03 19:18:29|         126|         116|   null|
|           HV0003|              B02875|2021-02-04 17:57:59|2021-02-04 18:17:56|